# CommomLit Readabilioty Prize
Kaggle의 Competition 중 하나인 Common Readability Prize를 풀어보고자 한다.
https://www.kaggle.com/c/commonlitreadabilityprize

In [1]:
import pandas as pd
train_df = pd.read_csv('C:/Users/USER/Desktop/Data_Practices/CommonLit Readability Prize/train.csv')
test_df = pd.read_csv('C:/Users/USER/Desktop/Data_Practices/CommonLit Readability Prize/test.csv')

In [2]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2829,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2830,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2831,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2832,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


## 문제 파악하기 및 변수의 의미

- id : 지문 고유 아이디
- url_legal : 출처 주소
- license : 라이센스
- excerpt : 분석 대상이 될 지문
- target : 난이도로, 값이 높을 수록 쉬운 지문이다.
- standard_error : 난이도를 매긴 사람들이 매긴 난이도의 편차가 어느 정도인가?

## EDA

In [3]:
train_df.isna().sum()

id                   0
url_legal         2004
license           2004
excerpt              0
target               0
standard_error       0
dtype: int64

In [4]:
test_df.isna().sum()

id           0
url_legal    4
license      4
excerpt      0
dtype: int64

지문의 자체 난이도가 어떤 출처의 유뮤나 라이센스와는 상관이 없을 것이므로, 지문을 제외하고는 날려도 무방할 것이다.

In [5]:
train_df = train_df[['excerpt', 'target', 'standard_error']]
test_df = test_df[['excerpt']]

In [6]:
train_df

,excerpt,target,standard_error
0,When the young people returned to the ballroom...,-0.340259,0.464009
1,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,And outside before the palace a great garden w...,-1.054013,0.450007
4,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...
2829,When you think of dinosaurs and where they liv...,1.711390,0.646900
2830,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2831,The second state of matter we will discuss is ...,0.255209,0.483866
2832,Solids are shapes that you can actually touch....,-0.215279,0.514128


In [7]:
train_df.iloc[0].excerpt

'When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThese rugs were for prizes, one for the girls and one for the boys. And this was the game.\nThe girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.\nThis would have been an easy matter, but each traveller was obliged to wear snowshoes.'

지문이 언제 어려울지를 고려하면 다음과 같다.
- 단어가 어려울경우
- 지문 자체가 긴 경우
- 하나의 문장이 길어 구조가 어려울 경우
- 문장 자체도 많은 경우

여러 요소를 반영하여 지문의 난이도를 측정해보고자 한다.